In [ ]:
from thinkbayes import Pmf

In [ ]:
class Cookie(Pmf):
    # assign the same prior to each hypo
    def __init__(self,hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo,1)
        self.Normalize()
    
    def Update(self,data):
        for hypo in self.Values():
            like = self.Likelihood(data,hypo)
            self.Mult(hypo,like)
        self.Normalize()
        
    mixes = {
        'Bowl1': dict(vanilla=0.75, chocolate=0.25),
        'Bowl2': dict(vanilla=0.5, chocolate=0.5)
    }
    
    def Likelihood(self, data, hypo):
        mix = self.mixes.get(hypo)
        like = mix[data]
        return like

In [ ]:
hypos = ['Bowl1', 'Bowl2']
pmf = Cookie(hypos)

In [ ]:
pmf.Update('vanilla')

In [ ]:
for hypo, prob in pmf.Items():
    print hypo, prob

In [ ]:
class Monty(Pmf):
    def __init__(self,hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo,1)
        self.Normalize()
    
    def Update(self,data):
        size = len(self.Values())
        for hypo in self.Values():
            like = self.Likelihood(data,hypo,size)
            self.Mult(hypo,like)
        self.Normalize()
    
    def Likelihood(self, data, hypo,size):
        if hypo == "B":
            like = 0
        elif hypo == "A":
            like = (size - 2)/float((size - 1))
        else:
            like = 1
        return like

In [ ]:
hypos = 'ABCDE'
pmf = Monty(hypos)
print pmf.Values()

In [ ]:
pmf.Update('B')

In [ ]:
for hypo, prob in pmf.Items():
    print hypo, prob

## Use Suite to unify the model

In [ ]:
from thinkbayes import Suite

In [ ]:
class Monty(Suite):
    # define the likelihood function of this problem
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

In [ ]:
suite=Monty('ABC')
suite.Update('B')
suite.Print()

In [ ]:
class M_and_M(Suite):
    mix94 = dict(brown=30, yellow=20, red=20, green=10,orange=10, tan=10)
    mix96= dict(blue=24, yellow=14, red=13, green=20,orange=16, brown=13)
    hypoA = dict(bag1 = mix94, bag2=mix96)
    hypoB = dict(bag1 = mix96, bag2=mix94)
    hypotheses = dict(A=hypoA, B=hypoB)
    def Likelihood(self, data, hypo):
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

In [ ]:
suite = M_and_M('AB')
suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))
suite.Print()

## Chapter 3

In [ ]:
class Dice(Suite):
    def Likelihood(self, data, hypo):
        # if the dice number is more than the number of sides of a specific dice, the likelihood is 0
        if hypo < data:
            return 0
        else:
            return 1.0/hypo

In [ ]:
suite = Dice([4,6,8,12,20])

In [ ]:
for roll in [6,8,7,7,5,4]:
    suite.Update(roll)
suite.Print()

## Locomotive problem

In [ ]:
hypos = xrange(1,1001)
class Train(Suite):
    def Likelihood(self, data, hypo):
        if hypo < data:
            return 0
        else:
            return 1.0/hypo
suite = Train(hypos)
for data in [30,60,90]:
    suite.Update(data)

In [ ]:
def Mean(suite):
    total = 0
    for hypo, prob in suite.Items():
        total += hypo*prob
    return total
print Mean(suite)

import thinkplot
thinkplot.PrePlot(1)
thinkplot.Pmf(suite)
thinkplot.Save(root='train1',
               xlabel='Number of trains',
               ylabel='Probability',
               formats=['pdf', 'eps'])

### Power-law distributed prior

In [ ]:
class Train(Dice):
    def __init__(self, companies, alpha = 1.0):
        Pmf.__init__(self)
        companies.sort()
        all_trains = [0] * (companies[-1] + 1)
        for company in companies:
            for x in range(1,company+1):
                all_trains[x] += 1
        for hypo in range(1,companies[-1]):
            self.Set(hypo, hypo**(-alpha) * all_trains[hypo])
        self.Normalize()

In [ ]:
companies = [150, 200, 100]
suite = Train(companies)
for data in [30,60,90]:
    suite.Update(data)
print Mean(suite)
thinkplot.PrePlot(1)
thinkplot.Pmf(suite)
# thinkplot.Save(root='train1',
#                xlabel='Number of trains',
#                ylabel='Probability',
#                formats=['pdf', 'eps'])
thinkplot.plot(suite)

In [ ]:
def Percentile(pmf, percentage):
    p = percentage / 100.0
    total = 0
    for val, prob in pmf.Items():
        total += prob
        if total >= p:
            return val
interval = Percentile(suite, 5), Percentile(suite, 95)
print interval

In [ ]:
cdf = suite.MakeCdf()
interval = cdf.Percentile(5), cdf.Percentile(95)
print interval